## LDA from scratch - Following Grus' code 

Simple example of topic modeling using LDA and Gibbs sampling. Each document has a 
districution of topics, and each topic has a distribution of weights. The idea is to 
use Gibbs sampling to get the join probability distributions of topics/words.

In [77]:
#Function to sample index according to input weights (probabilities). Needed for Gibbs sampling.
import random
def sample_from(weights):
    num = sum(weights)*random.random()
    c = 0
    for i,x in enumerate(weights):
        c += x
        if num < c:
            return i
    return i

In [78]:
#Testing sampling function
from collections import Counter
test = []
for _ in range(1000):
    test.append(sample_from([1,1,3]))
print Counter(test)

Counter({2: 595, 0: 206, 1: 199})


In [79]:
#Following Grus' example. Let's try to group these "documents" in 4 topics

documents = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

#Number of topics:
K = 4

### Now some definitions for the problem. 

- Each document has a certain distribution of topics. This distribution depends on the words that each document has. So, we need to count the number of times that a certain topic appears in a document.
- Further, a given word may belong to different topics, appearing more or less frequently. Then, the probability of a word belonging to a certain topic is calculated as the ratio of the times that word appear in a topic divided by the total number of words per topic?
- 

In [80]:
#Definitions below:

#How many times each topic is assigned to each document?
document_topic_counts = [Counter() for _ in documents]

#How many times each word is assigned to each topic?
topic_word_counts = [Counter() for _ in range(K)]

# Total number of words contained per topic:
topic_counts = [0 for _ in range(K)]

#Total number of words per document:
document_lengths = map(len, documents)

# Number of distict words:
distinct_words = set(word for document in documents for word in document)
W = len(distinct_words)

#Number of documents
D = len(documents)

### Now, we calculate the conditional probabilities of: topic|document and of word|topic.

p(topic|document) = # times topic appears in doc/total number of words in a document
p(word|topic) = # times word appears in a topic/total words in topic 

In [62]:
#Define conditional probability functions:
def p_topic_given_document(topic, d, alpha=0.1):  #p(t|D), using smoothing of 0.1 like in Naive Bayes
    return ((document_topic_counts[d][topic]+alpha)/(document_lengths[d]+K*alpha)) #

def p_word_given_topic(word, topic, beta=0.1):  #p(w|t), using smoothing of 0.1 like in Naive Bayes
    return ((topic_word_counts[topic][word]+beta)/(topic_counts[topic]+W*beta)) #

In [ ]:
#Use these probabilities to create the weights to update topics:

def topic_weight(d, word, k):
    #return weight for the k-topic given doc and word
    return p_word_given_topic(word, k)*p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

In [81]:
#Start by randomly assigning every word to a random topic:
random.seed(0)
document_topics = [[random.randrange(K) for word in document] for document in documents]

In [94]:
document_topics #How the topics were assigned, from 0 to 3

[[3, 3, 1, 1, 2, 1, 3],
 [1, 1, 2, 3, 2],
 [1, 3, 2, 1, 3, 3],
 [3, 3, 1, 2, 3],
 [2, 1, 0, 1],
 [2, 3, 3, 1, 3, 1],
 [3, 2, 0, 2],
 [1, 3, 2, 0],
 [1, 3, 0, 1],
 [3, 0, 2, 0],
 [3, 3, 1],
 [0, 1, 2, 3],
 [0, 2, 2],
 [2, 3, 2, 3, 2],
 [2, 1, 2]]

In [96]:
#Populate counters according with the randomly assigned topic per word:
for d in range(D): #For each doc...
    for word, topic in zip(documents[d], document_topics[d]): #For each word, topic in tuple (word, topic), populate...
        document_topic_counts[d][topic] += 1  #document d, topic 'topic'
        topic_word_counts[topic][word] += 1   #topic 'topic', word 'word'
        topic_counts[topic] += 1          #add topic 'topic' count

In [100]:
#Sample of counters
print document_topic_counts[4]
print topic_word_counts[0]
print topic_counts

Counter({1: 10, 0: 5, 2: 5})
Counter({'Big Data': 5, 'decision trees': 3, 'Java': 3, 'neural networks': 3, 'C++': 3, 'mathematics': 3, 'pandas': 3})
[40, 90, 95, 110]


## Now, use the preliminary topic allocations and Gibbs sampling to determine the final topic allocations.

Now, this is the neat part. We have a preliminary combo of (word, topic) per document. This allocation was done randomly with an uniform distribution from 0 to K. Using Gibbs sampling and the conditional p(t|d) and p(w|t), we can iteratively recalculate the (word,topic) combos such that the final values are consistent with the conditional probabilities as defined.

In [108]:
for ite in range(10000): 
    for d in range(D): #For all documents
        for i, (word, topic) in enumerate(zip(documents[d], document_topics[d])): #Iterate over (word, topic) for each document
            
            document_topic_counts[d][topic] -= 1  #substract one from current document/topic
            topic_word_counts[topic][word] -= 1 #substract one from current topic/word
            topic_counts[topic] -= 1 
            document_lengths[d] -= 1
            
            #Randomly choose new topic according to new weights: Weight kth = p_word_given_topic(word, k)*p_topic_given_document(k, d)
            #this is the "magic" part. We assume that the probability of a given word belonging to a topic equals the prior probability
            #of p(w|t) times p(t|d)
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic #reasign new topic to ith word of dth document
            
            document_topic_counts[d][new_topic] += 1 #increase topic count again
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

### Let's see now the most common words per topic:

In [117]:
for top_num, word_counts in enumerate(topic_word_counts):
    for word, count in word_counts.most_common()[:5]:
        print 'topic = ' + str(top_num) + ' , ' + 'word = ' + str(word) + ', counts = ' + str(count)

topic = 0 , word = Big Data, counts = 5
topic = 0 , word = neural networks, counts = 4
topic = 0 , word = mathematics, counts = 3
topic = 0 , word = pandas, counts = 3
topic = 0 , word = C++, counts = 3
topic = 1 , word = regression, counts = 6
topic = 1 , word = HBase, counts = 4
topic = 1 , word = statsmodels, counts = 4
topic = 1 , word = NoSQL, counts = 3
topic = 1 , word = Storm, counts = 3
topic = 2 , word = Postgres, counts = 5
topic = 2 , word = Python, counts = 5
topic = 2 , word = probability, counts = 4
topic = 2 , word = MongoDB, counts = 4
topic = 2 , word = Cassandra, counts = 4
topic = 3 , word = R, counts = 7
topic = 3 , word = Java, counts = 5
topic = 3 , word = statistics, counts = 5
topic = 3 , word = scikit-learn, counts = 5
topic = 3 , word = probability, counts = 4
